In [ ]:
!git clone https://github.com/advadj67/hw3_M11221004.git

Cloning into 'hw3_M11221004'...
remote: Enumerating objects: 13628, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 13628 (delta 16), reused 21 (delta 6), pack-reused 13594
Receiving objects: 100% (13628/13628), 1.64 GiB | 36.78 MiB/s, done.
Resolving deltas: 100% (3421/3421), done.
Updating files: 100% (13680/13680), done.


In [ ]:
!git clone https://github.com/3a832069/ML03_dataset.git

Cloning into 'ML03_dataset'...
remote: Enumerating objects: 6804, done.
remote: Total 6804 (delta 0), reused 0 (delta 0), pack-reused 6804
Receiving objects: 100% (6804/6804), 1.55 GiB | 13.89 MiB/s, done.
Resolving deltas: 100% (3402/3402), done.
Updating files: 100% (6843/6843), done.


In [ ]:
!git clone https://github.com/3a832069/YOLO_pr.git

Cloning into 'YOLO_pr'...
remote: Enumerating objects: 10212, done.
remote: Total 10212 (delta 0), reused 0 (delta 0), pack-reused 10212
Receiving objects: 100% (10212/10212), 1.48 GiB | 39.93 MiB/s, done.
Resolving deltas: 100% (3406/3406), done.
Updating files: 100% (13666/13666), done.


In [ ]:
#資料夾準備
import os
# 定義基礎路徑
base_path = '/content/ML03_dataset/貨櫃資料集'
subfolders = ['train', 'test', 'val']

# 創建images和labels資料夾
images_path = os.path.join(base_path, 'images')
labels_path = os.path.join(base_path, 'labels')

os.makedirs(images_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

# 在images和labels資料夾下創建train, test, val子資料夾
for subfolder in subfolders:
    os.makedirs(os.path.join(images_path, subfolder), exist_ok=True)
    os.makedirs(os.path.join(labels_path, subfolder), exist_ok=True)





In [ ]:
!pip install Pillow


In [ ]:
from PIL import Image
import os

# 定義路徑
base_path = '/content/ML03_dataset/貨櫃資料集'
input_folders = {
    'train': '訓練集',
    'test': '測試集',
    'val': '驗證集'
}
output_base_path = os.path.join(base_path, 'images')

# 定義函數來調整圖片大小並保存
def resize_and_save(input_folder, output_folder, size=(416, 416)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        if os.path.isfile(img_path):
            with Image.open(img_path) as img:
                img = img.resize(size, Image.ANTIALIAS)
                img.save(os.path.join(output_folder, img_name))

# 對每種類型的資料進行處理
for image_type, folder_name in input_folders.items():
    input_folder = os.path.join(base_path, folder_name)
    output_folder = os.path.join(output_base_path, image_type)
    resize_and_save(input_folder, output_folder)

print("所有圖片已成功調整大小並另存。")


<ipython-input-3-a2a289944364>:21: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(size, Image.ANTIALIAS)


所有圖片已成功調整大小並另存。


In [ ]:
from PIL import Image
import os

# 定義路徑
original_train_path = '/content/ML03_dataset/貨櫃資料集/測試集'
resized_train_path = '/content/ML03_dataset/貨櫃資料集/images/test'

# 定義函數來打印圖片的shape
def print_image_shapes(folder_path, num_images=2):
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    for img_name in image_files[:num_images]:
        img_path = os.path.join(folder_path, img_name)
        with Image.open(img_path) as img:
            print(f"Image: {img_name}, Shape: {img.size}")

# 打印原始圖片的shape
print("Original Images:")
print_image_shapes(original_train_path)

# 打印調整大小後的圖片的shape
print("Resized Images:")
print_image_shapes(resized_train_path)


Original Images:
Image: image_0667.jpg, Shape: (1280, 720)
Image: image_0017.jpg, Shape: (1920, 1080)
Resized Images:
Image: image_0667.jpg, Shape: (416, 416)
Image: image_0017.jpg, Shape: (416, 416)


In [ ]:
#標籤處理
import os
import xml.etree.ElementTree as ET

classes = ['container']

# 標籤處理(符合YOLO格式)
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_xml_to_yolo(xmlPath, output_folder):
    tree = ET.parse(xmlPath)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    # 建立YOLO文件
    txt_filename = os.path.splitext(os.path.basename(xmlPath))[0] + '.txt'
    with open(os.path.join(output_folder, txt_filename), "w") as bbox:

        for obj in root.iter('object'):
            difficult = obj.find('difficult').text
            cls = obj.find('name').text
            if cls not in classes or int(difficult) == 1:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text),
                 float(xmlbox.find('xmax').text),
                 float(xmlbox.find('ymin').text),
                 float(xmlbox.find('ymax').text))

            # 轉換成 YOLO 格式
            converted_bb = convert((w, h), b)
            bbox.write(f"{cls_id} {' '.join(str(coord) for coord in converted_bb)}\n")

# 處理整個資料夾
def process_xml_folder(xml_folder, output_folder):
    xml_files = [f for f in os.listdir(xml_folder) if f.endswith('.xml')]
    for xml_file in xml_files:
        xml_path = os.path.join(xml_folder, xml_file)
        convert_xml_to_yolo(xml_path, output_folder)

# 資料夾路徑設置
base_path = '/content/ML03_dataset/貨櫃資料集'
train_xml_folder = os.path.join(base_path, '訓練集_xml')
output_train_folder = os.path.join(base_path, 'labels/train')

val_xml_folder = os.path.join(base_path, '驗證集_xml')
output_val_folder = os.path.join(base_path, 'labels/val')

test_xml_folder = os.path.join(base_path, '測試集_xml')
output_test_folder = os.path.join(base_path, 'labels/test')

# 確認已建立
os.makedirs(output_train_folder, exist_ok=True)
os.makedirs(output_val_folder, exist_ok=True)
os.makedirs(output_test_folder, exist_ok=True)

# 進行格式轉換
process_xml_folder(train_xml_folder, output_train_folder)
process_xml_folder(val_xml_folder, output_val_folder)
process_xml_folder(test_xml_folder, output_test_folder)

print('標籤準備完成')


標籤準備完成


In [ ]:
#建立 v5data.yaml
# 定義v5data.yaml文件內容
yaml_content = """# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /content/ML03_dataset/貨櫃資料集  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test: images/test  # test images (optional)

names:
  0: "container number"
"""

#定義路徑
yaml_file_path = '/content/ML03_dataset/貨櫃資料集/v5data.yaml'

# 寫入文件
with open(yaml_file_path, 'w') as file:
    file.write(yaml_content)



In [ ]:
#建立 yolov5.yaml
# 定義yolov5.yaml文件內容
yaml_content = """# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLOv5 object detection model with P3-P5 outputs. For details see https://docs.ultralytics.com/models/yolov5

# Parameters
nc: 1 # number of classes
names: "container number"

scales: # model compound scaling constants, i.e. 'model=yolov5n.yaml' will call yolov5.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024]
  s: [0.33, 0.50, 1024]
  m: [0.67, 0.75, 1024]
  l: [1.00, 1.00, 1024]
  x: [1.33, 1.25, 1024]

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  - [-1, 1, Conv, [64, 6, 2, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C3, [128]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 6, C3, [256]]
  - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
  - [-1, 9, C3, [512]]
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C3, [1024]]
  - [-1, 1, SPPF, [1024, 5]] # 9

# YOLOv5 v6.0 head
head:
  - [-1, 1, Conv, [512, 1, 1]]
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C3, [512, False]] # 13

  - [-1, 1, Conv, [256, 1, 1]]
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C3, [256, False]] # 17 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 14], 1, Concat, [1]] # cat head P4
  - [-1, 3, C3, [512, False]] # 20 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 10], 1, Concat, [1]] # cat head P5
  - [-1, 3, C3, [1024, False]] # 23 (P5/32-large)

  - [[17, 20, 23], 1, Detect, [nc]] # Detect(P3, P4, P5)
"""

#定義路徑
yaml_file_path = '/content/ML03_dataset/yolov5.yaml'

# 寫入文件
with open(yaml_file_path, 'w') as file:
    file.write(yaml_content)

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.9/756.9 kB 16.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov5n.yaml').load('yolov5n.pt')  # build from YAML and transfer weights


results = model.train(data='/content/ML03_dataset/貨櫃資料集/v5data.yaml', epochs=8 , imgsz=416)


best_model_path = "/content/best_v5model.pt"

model.save(best_model_path)

model.val()


PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 5.27M/5.27M [00:00<00:00, 235MB/s]

Transferred 427/427 items from pretrained weights


Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov5n.yaml, data=/content/ML03_dataset/貨櫃資料集/v5data.yaml, epochs=8, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 135MB/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  3.29it/s]


                   all        536        536          1          1      0.995      0.863
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train
Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv5n summary (fused): 193 layers, 2503139 parameters, 0 gradients, 7.1 GFLOPs


val: Scanning /content/ML03_dataset/貨櫃資料集/labels/val.cache... 536 images, 0 backgrounds, 0 corrupt: 100%|██████████| 536/536 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:07<00:00,  4.46it/s]


                   all        536        536          1          1      0.995      0.864
Speed: 0.2ms preprocess, 4.0ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79afa0706080>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
'''
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data='/content/ML03_dataset/貨櫃資料集/v8data.yaml', epochs=8, patience=5, imgsz=416)

# Save the best model
best_model_path = "/content/best_v8model.pt"
model.save(best_model_path)

model.val()  # It'll automatically evaluate the data you trained.
'''

In [ ]:
import shutil

# 指定要下载的文件夹路径
folder_path = "/content/runs"

# 将文件夹压缩成zip文件
shutil.make_archive("/content/runs", 'zip', folder_path)

# 下载压缩文件
from google.colab import files
files.download("/content/runs.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#圖像預測
from ultralytics import YOLO
from PIL import Image
import cv2
import os

# 定義來源資料夾和結果儲存資料夾
source_folder = '/content/ML03_dataset/貨櫃資料集/images/test'
output_folder = '/content/pre_results'

# 確保結果儲存資料夾存在
os.makedirs(output_folder, exist_ok=True)

# 遍歷資料夾中的所有圖像檔案
for filename in os.listdir(source_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # 可根據需要添加更多的圖像格式
        source_path = os.path.join(source_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # 執行預測
        results = model(source_path)

        # 顯示物件類別和座標
        print(f'Image: {filename}')
        print('Classes:', results[0].boxes.cls)
        print('Coordinates:', results[0].boxes.xyxy)
        print()

        # 保存預測結果的圖像
        # 使用cv2讀取和保存圖像以保留檔案名
        img = cv2.imread(source_path)
        result_img = results[0].plot()  # 繪製預測結果
        cv2.imwrite(output_path, result_img)

print('所有圖像預測完成，結果已保存到:', output_folder)


串流輸出內容已截斷至最後 5000 行。


image 1/1 /content/ML03_dataset/貨櫃資料集/images/test/image_0425.jpg: 416x416 1 container number, 9.6ms
Speed: 0.8ms preprocess, 9.6ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 416)
Image: image_0425.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[199.5721,  84.3282, 238.4923, 112.3428]], device='cuda:0')


image 1/1 /content/ML03_dataset/貨櫃資料集/images/test/image_0468.jpg: 416x416 1 container number, 9.6ms
Speed: 0.7ms preprocess, 9.6ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 416)
Image: image_0468.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[142.8985, 154.3812, 234.2346, 216.0565]], device='cuda:0')


image 1/1 /content/ML03_dataset/貨櫃資料集/images/test/image_0407.jpg: 416x416 1 container number, 9.0ms
Speed: 0.7ms preprocess, 9.0ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 416)
Image: image_0407.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[172.5556, 209.0753

In [ ]:
import shutil

# 指定要下载的文件夹路径
folder_path = '/content/pre_results'

# 将文件夹压缩成zip文件
shutil.make_archive("/content/pre_results", 'zip', folder_path)

# 下载压缩文件
from google.colab import files
files.download("/content/pre_results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#圖像預測和績效
from ultralytics import YOLO
from PIL import Image
import cv2
import os
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, average_precision_score

# 定義來源資料夾和結果儲存資料夾
source_folder = '/content/ML03_dataset/貨櫃資料集/images/test'
output_folder = '/content/pre_results'
labels_folder = '/content/ML03_dataset/貨櫃資料集/labels/test'  # 替換為你的標籤文件夾路徑

# 確保結果儲存資料夾存在
os.makedirs(output_folder, exist_ok=True)

# 初始化變量
all_true_boxes = []
all_pred_boxes = []
all_scores = []
all_classes = []

# 遍歷資料夾中的所有圖像檔案
for filename in os.listdir(source_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # 可根據需要添加更多的圖像格式
        source_path = os.path.join(source_folder, filename)
        output_path = os.path.join(output_folder, filename)
        label_path = os.path.join(labels_folder, os.path.splitext(filename)[0] + '.txt')

        # 執行預測
        results = model(source_path)

        # 顯示物件類別和座標
        print(f'Image: {filename}')
        print('Classes:', results[0].boxes.cls)
        print('Coordinates:', results[0].boxes.xyxy)
        print()

        # 保存預測結果的圖像
        img = cv2.imread(source_path)
        result_img = results[0].plot()  # 繪製預測結果
        cv2.imwrite(output_path, result_img)

        # 讀取真實標籤
        true_boxes = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                true_boxes.append([class_id, x_center, y_center, width, height])

        # 將真實標籤轉換為與預測標籤相同的格式
        img_h, img_w, _ = img.shape
        true_boxes = np.array(true_boxes)
        true_boxes[:, 1:3] = true_boxes[:, 1:3] * [img_w, img_h]
        true_boxes[:, 3:5] = true_boxes[:, 3:5] * [img_w, img_h]
        true_boxes[:, 1:3] -= true_boxes[:, 3:5] / 2
        true_boxes[:, 3:5] += true_boxes[:, 1:3]

        all_true_boxes.extend(true_boxes)

        # 收集預測結果
        for box in results[0].boxes:
            all_pred_boxes.append(box.xyxy.cpu().numpy())
            all_scores.append(box.conf.cpu().numpy())
            all_classes.append(box.cls.cpu().numpy())

# 計算 precision, recall, f1-score, mAP
y_true = np.array([box[0] for box in all_true_boxes])
y_pred = np.array(all_classes)
y_scores = np.array(all_scores)

precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
average_precision = average_precision_score(y_true, y_scores, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')
print(f'mAP: {average_precision}')


串流輸出內容已截斷至最後 5000 行。
Speed: 0.9ms preprocess, 10.2ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 416)
Image: image_0425.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[199.5721,  84.3282, 238.4923, 112.3428]], device='cuda:0')


image 1/1 /content/ML03_dataset/貨櫃資料集/images/test/image_0468.jpg: 416x416 1 container number, 9.7ms
Speed: 0.8ms preprocess, 9.7ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 416)
Image: image_0468.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[142.8985, 154.3812, 234.2346, 216.0565]], device='cuda:0')


image 1/1 /content/ML03_dataset/貨櫃資料集/images/test/image_0407.jpg: 416x416 1 container number, 13.1ms
Speed: 0.8ms preprocess, 13.1ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 416)
Image: image_0407.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[172.5556, 209.0753, 282.8716, 281.1096]], device='cuda:0')


image 1/1 /content/ML03_dataset/貨櫃資料集/images/test/image_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [ ]:
# 图像预测和績效(2)
from ultralytics import YOLO
from PIL import Image
import cv2
import os
import numpy as np
import torch
from ultralytics.utils.metrics import ConfusionMatrix, Metric

# 定义来源文件夹和结果保存文件夹
source_folder = '/content/ML03_dataset/貨櫃資料集/images/test'
output_folder = '/content/pre_results'
labels_folder = '/content/ML03_dataset/貨櫃資料集/labels/test'  # 替换为你的标签文件夹路径

# 确保结果保存文件夹存在
os.makedirs(output_folder, exist_ok=True)

# 初始化混淆矩阵和指标计算类
nc = 1  # 类别数量，根据实际情况设置
conf_matrix = ConfusionMatrix(nc=nc, task='detect')
metrics = Metric()

# 遍历文件夹中的所有图像文件
for filename in os.listdir(source_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # 可根据需要添加更多的图像格式
        source_path = os.path.join(source_folder, filename)
        output_path = os.path.join(output_folder, filename)
        label_path = os.path.join(labels_folder, os.path.splitext(filename)[0] + '.txt')

        # 执行预测
        results = model(source_path)

        # 显示对象类别和坐标
        print(f'Image: {filename}')
        print('Classes:', results[0].boxes.cls)
        print('Coordinates:', results[0].boxes.xyxy)
        print()

        # 保存预测结果的图像
        img = cv2.imread(source_path)
        result_img = results[0].plot()  # 绘制预测结果
        cv2.imwrite(output_path, result_img)

        # 读取真实标签
        true_boxes = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                true_boxes.append([class_id, x_center, y_center, width, height])

        # 将真实标签转换为与预测标签相同的格式
        img_h, img_w, _ = img.shape
        true_boxes = np.array(true_boxes)
        true_boxes[:, 1:3] = true_boxes[:, 1:3] * [img_w, img_h]
        true_boxes[:, 3:5] = true_boxes[:, 3:5] * [img_w, img_h]
        true_boxes[:, 1:3] -= true_boxes[:, 3:5] / 2
        true_boxes[:, 3:5] += true_boxes[:, 1:3]

        true_boxes_formatted = np.zeros((true_boxes.shape[0], 5))
        true_boxes_formatted[:, 0] = true_boxes[:, 0]  # class
        true_boxes_formatted[:, 1:] = true_boxes[:, 1:]  # x1, y1, x2, y2

        true_boxes_formatted = torch.tensor(true_boxes_formatted, dtype=torch.float32)

        # 收集预测结果并更新混淆矩阵
        pred_boxes = results[0].boxes.xyxy.cpu().numpy()
        pred_scores = results[0].boxes.conf.cpu().numpy()
        pred_classes = results[0].boxes.cls.cpu().numpy()

        # 将预测结果格式化为 [x1, y1, x2, y2, score, class]
        detections = np.concatenate([pred_boxes, pred_scores[:, None], pred_classes[:, None]], axis=1)
        detections = torch.tensor(detections, dtype=torch.float32)
        pred_classes = torch.tensor(pred_classes, dtype=torch.int64)

        conf_matrix.process_batch(detections, true_boxes_formatted, pred_classes)
        metrics.update((pred_boxes, pred_scores, pred_classes, true_boxes_formatted))

# 计算 precision, recall, f1-score, mAP
precision, recall, f1 = metrics.p, metrics.r, metrics.f1
map50 = metrics.map50
map = metrics.map

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')
print(f'mAP50: {map50}')
print(f'mAP: {map}')



image 1/1 /content/ML03_dataset/貨櫃資料集/images/test/image_0667.jpg: 416x416 1 container number, 17.6ms
Speed: 1.3ms preprocess, 17.6ms inference, 2.1ms postprocess per image at shape (1, 3, 416, 416)
Image: image_0667.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[192.4337, 123.9721, 256.7152, 168.1556]], device='cuda:0')



RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 2

In [ ]:
print(1)

1


In [14]:
#OCR (待處理)
import cv2
import pytesseract
from google.colab.patches import cv2_imshow
import torch

# 定義裁切並進行文字辨識的函數
def crop_and_recognize(image, bounding_box):
    # 檢查image是否為None
    if image is None:
        print("Error: Failed to read the image.")
        return

    # 將tensor轉換為Python列表並取整數
    bounding_box = bounding_box.cpu().detach().numpy().astype(int).tolist()[0]  # 注意加上[0]以取得單一元素
    # 裁剪框框的區域
    x1, y1, x2, y2 = bounding_box
    cropped_image = image[y1:y2, x1:x2]
    cv2_imshow(cropped_image)
    # 將裁剪後的圖片轉換為灰階
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    # 使用大津法計算閾值
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # 反轉二值化圖片，將背景設置為白色，文字設置為黑色
    inverted_image = cv2.bitwise_not(binary_image)

    # 設置文字辨識引擎（使用Tesseract OCR）
    pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

    # 進行文字辨識
    text = pytesseract.image_to_string(inverted_image)

    # 顯示裁切後的圖片
    cv2_imshow(inverted_image)

    # 顯示辨識的結果
    print("Text in box:", text)

# 要處理的圖片路徑
image_path = '/content/pre_results/image_0425.jpg'

# 讀取圖片
image = cv2.imread(image_path)

# 單個框框的座標（這裡使用你提供的tensor表示）
bounding_box = torch.tensor([[199.5721, 84.3282, 238.4923, 112.3428]], device='cuda:0')

# 裁切並進行文字辨識
crop_and_recognize(image, bounding_box)


Text in box: 


In [1]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract opencv-python-headless


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,489 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
'''
# 績效評估
from ultralytics import YOLO
from ultralytics.utils.metrics import Metric
import os

# Initialize a Metric object
metric = Metric()

# 載入最佳模型
best_model_path = "/content/best_v5model.pt"
best_model = YOLO('yolov5n.yaml').load(best_model_path)

# 定義測試集的圖像路徑
test_image_path = '/content/ML03_dataset/貨櫃資料集/images/test'

# 定義測試集的標籤路徑
test_label_path = '/content/ML03_dataset/貨櫃資料集/labels/test'

# 加載所有標籤文件的內容
ground_truth_annotations = []

for label_file in os.listdir(test_label_path):
    if label_file.endswith('.txt'):
        with open(os.path.join(test_label_path, label_file), 'r') as f:
            lines = f.readlines()
            annotations = []
            for line in lines:
                # 解析每一行標籤，並將其加入到 annotations 列表中
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                x_min = max(0, (x_center - width / 2))
                y_min = max(0, (y_center - height / 2))
                x_max = min(1, (x_center + width / 2))
                y_max = min(1, (y_center + height / 2))
                annotations.append({
                    'class_id': int(class_id),
                    'bbox': [x_min, y_min, x_max, y_max]
                })
            ground_truth_annotations.append(annotations)

# 進行模型在測試集上的預測
model_predictions = {}

for img_file in os.listdir(test_image_path):
    img_path = os.path.join(test_image_path, img_file)
    results = best_model(img_path)
    predictions = []
    for result in results:
        for box in result.boxes.xyxy:
            class_id = int(box[0])
            confidence = float(box[4])
            x_min, y_min, x_max, y_max = box[1:5]
            predictions.append({
                'class_id': class_id,
                'confidence': confidence,
                'bbox': [x_min, y_min, x_max, y_max]
            })
    model_predictions[img_path] = predictions

# 更新評估指標
metric.update(ground_truth_annotations, model_predictions)

# 計算評估指標
map50 = metric.map50()  # Mean Average Precision at IoU threshold of 0.5
map75 = metric.map75()  # Mean Average Precision at IoU threshold of 0.75
map_ = metric.map()     # Mean Average Precision at IoU thresholds from 0.5 to 0.95

# 輸出評估結果
print("mAP@0.5:", map50)
print("mAP@0.75:", map75)
print("mAP@0.5:0.95:", map_)
'''

Transferred 427/427 items from pretrained weights

image 1/1 /content/ML03_dataset/貨櫃資料集/images/test/image_0540.jpg: 640x640 3 2s, 2 6s, 15.3ms
Speed: 2.4ms preprocess, 15.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


IndexError: index 4 is out of bounds for dimension 0 with size 4